# Create object LBP signatures - after frequency counted has been made into numpy
This code attempts to combine the LBP numbers for (by doing a frequency count across them) individual object (simulated nuclei). This one is doing this with one property difference

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import math
from PIL import Image
#import cv2
import skimage
import tifffile
import os
import pandas as pd
#import napari
#print(napari.__version__)
#from dask_image.imread import imread
#import dask.array as da
#import io

import platform
if platform.system() == 'Windows':
    fileroot = 'X:/'
    print('Loading Windows')
if platform.system() == 'Linux':
    fileroot = '/home/jovyan/'
    print('Loading Linux')
if os.path.exists('/home/ubuntu/Documents/'):
    fileroot = '/home/ubuntu/Documents/'
if os.path.exists('/nfs/team283_imaging/') and not os.path.exists('/home/jovyan/'):
    fileroot = '/nfs/team283_imaging/'

colors_new=['#0072b2','#d55e00','#009e73', '#cc79a7','#f0e442','#56b4e9']

import sys
sys.path.append(fileroot+'VU_TEX/Ben_utilities/')
import useful_functions as uf
import numba_funcs as nf

Loading Linux


In [3]:
fileroot

'/home/jovyan/'

In [4]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [5]:
maxi = 3328 #this is the maximum value that the LBP could get to

In [6]:
from tqdm import tqdm
from scipy import ndimage as ndi
from functools import partial
from copy import deepcopy
from itertools import product
from datetime import datetime
from tqdm.notebook import trange, tqdm
import anndata as ad

### Loading the data

In [7]:
patchsize = 100

In [8]:
directory_originals = [fileroot + 'VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-06d_image_conversion_and_crop_to_small_version/']
#directory_originals = [fileroot + 'team283_imaging/0ExternalData/2022-09-01_IVY_GAP/2022-08-31_originals/']
filenames_originals = ['V11J11-099__Z4_FO3_01__A1_small.tif']
#filenames_originals = os.listdir(directory_originals[0])
#filenames_originals = [file.replace('.npy', '.jpg') for file in filenames_originals]
#fullpaths_originals = [x + y for x,y in zip(directory_originals, filenames_originals)]
fullpaths_originals = [directory_originals[0] + y for y in filenames_originals if os.path.exists(directory_originals[0] + y)]
fullpaths_originals

['/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-06d_image_conversion_and_crop_to_small_version/V11J11-099__Z4_FO3_01__A1_small.tif']

In [9]:
directory_lbp_top = fileroot + 'VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07_LBP_parallel_values_for_small_image/'
print(directory_lbp_top)
#should be aligned with fullpaths_originals

/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07_LBP_parallel_values_for_small_image/


In [10]:
#directory_gt = fileroot + 'team283_imaging/0ExternalData/2022-09-01_IVY_GAP/2022-09-02_integer_annotations/'
#directory_gt = fileroot + 'VU_TEX/playground_Ben/2022-08-26_IVY_GAP/2022-11-09_Subset_optimisation_LBP_v18_patchsize_200/2022-11-09_integer_annotations_binned_patchsize_200/'
#filename_gts = os.listdir(directory_gt)
#filename_gts = [each.replace('.jpg', '_annotation.npy') for each in filenames_originals]
#fullpaths_gts = [directory_gt + filename_gt for filename_gt in filename_gts if os.path.exists(directory_gt + filename_gt)]
#fullpaths_gts

In [11]:
#directory_patches_indices = fileroot + 'VU_TEX/playground_Ben/2021-12-20_Thymus/2021-12-20b_Initial_image_conversions/2021-12-20_Patches/'
#filename_patches_indices = 'HnE_norm_V10S15_patchsize_200_indices_from_1.npy'
#fullpaths_patches_indices = [directory_patches_indices + filename_patches_indices]

In [12]:
directory_out = fileroot + 'VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07c_LBP_B_parallel_analysis_small_image/patchsize_100/'
filename_out_base = 'V11J11-099__Z4_FO3_01__A1_small_LBP_'

In [13]:
#directory_images_metadata = fileroot + 'team283_imaging/' + '0ExternalData/2022-09-01_IVY_GAP/'
#directory_images_metadata = fileroot + '0ExternalData/2022-09-01_IVY_GAP/'
#filename_images_metadata = '2022-09-01_IVY_GAP_metadata.csv'
#df_images_metadata = pd.read_csv(directory_images_metadata + filename_images_metadata, index_col=0)
#df_images_metadata

df_images_metadata = pd.DataFrame({'original_index': [0], 'output_filename':filenames_originals, 
                                 })
this_shape = skimage.io.imread(fullpaths_originals[0]).shape
df_images_metadata['image_width'] = this_shape[0]
df_images_metadata['image_height'] = this_shape[1]
df_images_metadata

,original_index,output_filename,image_width,image_height
0,0,V11J11-099__Z4_FO3_01__A1_small.tif,5539,3739


In [14]:
#df_images_metadata = df_images_metadata.loc[df_images_metadata['original_index'].isin([50, 594, 406, 418, 
#267, 290, 474, 231, 498, 217, 
#309, 544])]
#df_images_metadata

In [15]:
#directory_colors_metadata = fileroot + 'team283_imaging/' + '0ExternalData/2022-09-01_IVY_GAP/'
#directory_colors_metadata = fileroot + '0ExternalData/2022-09-01_IVY_GAP/'
#filename_colors_metadata = '2022-09-02_colors_for_IVY_GAP.csv'
#df_colors = pd.read_csv(directory_colors_metadata + filename_colors_metadata, index_col=0)
#df_colors

In [16]:
# make list of methods
methods_list = os.listdir(directory_lbp_top)
methods_list.sort(key=partial(uf.getvaluefromstringbest, variable='radius', mydtype=int))
methods_list.sort(key=partial(uf.getvaluefromstringbest, variable='Ch', mydtype=int))
methods_list = [each for each in methods_list if
               uf.getvaluefromstringbest(each, variable='radius', mydtype=int) < 200]
method_list_cols = []
for each in methods_list:
    print(each)
    this_npoints = uf.getvaluefromstringbest(each, variable='npoints', mydtype=int)
    for i in range(0, this_npoints+2):
        method_list_cols.append(each+'_value_'+str(i))
    print(this_npoints)
print(method_list_cols)

Ch_0_METHOD_uniform_npoints_7_radius_1
7
Ch_0_METHOD_uniform_npoints_13_radius_2
13
Ch_0_METHOD_uniform_npoints_19_radius_3
19
Ch_0_METHOD_uniform_npoints_26_radius_4
26
Ch_0_METHOD_uniform_npoints_32_radius_5
32
Ch_0_METHOD_uniform_npoints_38_radius_6
38
Ch_0_METHOD_uniform_npoints_51_radius_8
51
Ch_0_METHOD_uniform_npoints_70_radius_11
70
Ch_0_METHOD_uniform_npoints_88_radius_14
88
Ch_0_METHOD_uniform_npoints_120_radius_19
120
Ch_0_METHOD_uniform_npoints_158_radius_25
158
Ch_0_METHOD_uniform_npoints_214_radius_34
214
Ch_0_METHOD_uniform_npoints_283_radius_45
283
Ch_0_METHOD_uniform_npoints_371_radius_59
371
Ch_0_METHOD_uniform_npoints_497_radius_79
497
Ch_1_METHOD_uniform_npoints_7_radius_1
7
Ch_1_METHOD_uniform_npoints_13_radius_2
13
Ch_1_METHOD_uniform_npoints_19_radius_3
19
Ch_1_METHOD_uniform_npoints_26_radius_4
26
Ch_1_METHOD_uniform_npoints_32_radius_5
32
Ch_1_METHOD_uniform_npoints_38_radius_6
38
Ch_1_METHOD_uniform_npoints_51_radius_8
51
Ch_1_METHOD_uniform_npoints_70_radius_

In [17]:
len(method_list_cols)

6051

In [18]:
overall_number_of_patches = 0
for height, width in zip(df_images_metadata['image_height'], df_images_metadata['image_width']):
    height_sp = math.floor(height/patchsize)
    width_sp = math.floor(width/patchsize)
    overall_number_of_patches += height_sp*width_sp
overall_number_of_patches

2035

In [21]:
anndata_list = []
anndata_concat = ad.AnnData()
out_array = np.zeros((overall_number_of_patches, len(method_list_cols)), dtype=np.uint16)
obs_list = []
m = 0
n = 0

#for j, output_filename, output_fname_annotated in tqdm(zip(df_images_metadata['original_index'], 
#                                                      df_images_metadata['output_filename'],
#                                                     df_images_metadata['output_filename_annotations']),
#                                                      total = len(df_images_metadata)):
for j, output_filename in tqdm(zip(df_images_metadata['original_index'], 
                                                      df_images_metadata['output_filename'],
                                                     ),
                                                      total = len(df_images_metadata)):

#    print(j, output_filename, output_fname_annotated)
    
    fpath_for_shape = directory_lbp_top + methods_list[0] + '/' + output_filename.replace('.tif', '.npy')
    this_test_array = np.load(fpath_for_shape, mmap_mode='r')
    this_shape = this_test_array.shape
    this_dtype = this_test_array.dtype
    
    #this part creates the X0 and X1 coordinates
    x0_array = np.zeros((this_shape[0], this_shape[1]), dtype=np.uint16)
    for i0 in range(this_shape[0]):
        x0_array[i0] = i0
    
    x1_array = np.zeros((this_shape[0], this_shape[1]), dtype=np.uint16)
    for i1 in range(this_shape[1]):
        x1_array[:, i1] = i1
        
    #loading the groundtruth
#    gt = np.load(directory_gt + output_fname_annotated.replace('.jpg', '.npy'))
    
    output_array = np.zeros((this_shape[0], this_shape[1], len(method_list_cols)), dtype=this_dtype)
#    print(this_shape, this_dtype, output_array.shape)
    
    start_index = 0
    for each_method in methods_list:
        fpath_to_add = directory_lbp_top + each_method + '/' + output_filename.replace('.tif', '.npy')
        this_npoints = uf.getvaluefromstringbest(each_method, variable='npoints', mydtype=int)
        array_to_add = np.load(fpath_to_add)
        end_index = start_index + this_npoints + 2
        #add to the array
#        print(fpath_to_add, start_index, end_index, array_to_add.shape)
        output_array[:, :, start_index:end_index] = array_to_add
        start_index = end_index
    
    #this part reshapes the arrays
    reshaped = np.reshape(output_array, (-1, output_array.shape[2]))
    reshaped_x0 = np.reshape(x0_array, (-1))
    reshaped_x1 = np.reshape(x1_array, (-1))
#    reshaped_gt = np.reshape(gt, (-1))
    reshaped_gt = np.zeros(reshaped_x0.shape)
        
    this_AD = ad.AnnData(reshaped, var=method_list_cols, dtype=np.uint16)
    
    this_AD.obs['this_image_index'] = this_AD.obs.index
    this_AD.obs['X0'] = reshaped_x0 
    this_AD.obs['X1'] = reshaped_x1 
    this_AD.obs['Groundtruth'] = reshaped_gt
    this_AD.obs['original_index'] = j
    this_AD.obs['output_filename'] = output_filename
#    this_AD.obs['output_fname_annotated'] = output_fname_annotated
    
#    anndata_list.append(this_AD)
#    if m == 0:
#        anndata_concat = this_AD
#    else:
#        anndata_concat = ad.concat((anndata_concat, this_AD), axis=0)
#    break
    this_shape = this_AD.X.shape
    out_array[n:n+this_shape[0]] = this_AD.X
    print(n, n+this_shape[0])
    n+=this_shape[0]
    
    obs_list.append(this_AD.obs)

    m+=1

  0%|          | 0/1 [00:00<?, ?it/s]

0 2035


In [22]:
n

2035

In [23]:
out_array[n-1]

array([ 551,  564, 1100, ...,    0,    0, 9474], dtype=uint16)

In [25]:
try:
    out_array[n]
except:
    print('Index is out of bounds')

Index is out of bounds


In [26]:
out_array_shortened = out_array[0:n]
out_array_shortened

array([[ 789,  707, 1054, ...,    1,    4, 9360],
       [ 722,  643, 1001, ...,    0,    0, 9823],
       [ 619,  511, 1018, ...,   11,   35, 9716],
       ...,
       [ 541,  491, 1055, ...,    5,    8, 9731],
       [ 560,  546, 1003, ...,    3,   13, 9564],
       [ 551,  564, 1100, ...,    0,    0, 9474]], dtype=uint16)

In [27]:
#anndata_concat.X.shape

In [28]:
start = datetime.now(); print(start)
obs_concat = pd.concat(obs_list, ignore_index=True)
print(datetime.now()-start)

2023-07-07 08:12:56.785075
0:00:00.001112


In [29]:
obs_concat

,this_image_index,X0,X1,Groundtruth,original_index,output_filename
0,0,0,0,0.0,0,V11J11-099__Z4_FO3_01__A1_small.tif
1,1,0,1,0.0,0,V11J11-099__Z4_FO3_01__A1_small.tif
2,2,0,2,0.0,0,V11J11-099__Z4_FO3_01__A1_small.tif
3,3,0,3,0.0,0,V11J11-099__Z4_FO3_01__A1_small.tif
4,4,0,4,0.0,0,V11J11-099__Z4_FO3_01__A1_small.tif
...,...,...,...,...,...,...
2030,2030,54,32,0.0,0,V11J11-099__Z4_FO3_01__A1_small.tif
2031,2031,54,33,0.0,0,V11J11-099__Z4_FO3_01__A1_small.tif
2032,2032,54,34,0.0,0,V11J11-099__Z4_FO3_01__A1_small.tif
2033,2033,54,35,0.0,0,V11J11-099__Z4_FO3_01__A1_small.tif


In [30]:
#anndata_list[0].X

In [31]:
#anndata_list[1].X

In [32]:
#anndata_concat = ad.concat(anndata_list, axis=0, index_unique='-')
#anndata_concat.obs.reset_index(inplace=True, drop=True)
#anndata_concat.obs.index = anndata_concat.obs.index.astype(str)
#anndata_concat.obs = pd.DataFrame(anndata_concat.obs.index, columns=['Here'])
#anndata_concat.obs.index = anndata_concat.obs.index.astype(str)
#anndata_concat.obs['output_filename'] = anndata_concat.obs['output_filename'].astype(str)
#anndata_concat.var = pd.DataFrame(method_list_cols)
#display(anndata_concat)
#display(anndata_concat.var)
#display(anndata_concat.obs)

In [33]:
ad.__version__

'0.9.1'

In [34]:
start = datetime.now(); print(start)
np.save("2023-07-07_LBP_all_images_data_X.npy", out_array_shortened)
print(datetime.now()-start)

2023-07-07 08:12:56.908986
0:00:00.095228


In [35]:
obs_concat.to_csv("2023-07-07_LBP_all_images_data_OBS.csv")

In [36]:
pd.DataFrame(method_list_cols).to_csv("2023-07-07_LBP_all_images_data_VAR.csv")

In [37]:
#anndata_concat.write("2022-10-11_LBP_all_images_data.h5ad")